<a href="https://colab.research.google.com/github/Nskoushik26276/Product-Pulse-AI/blob/main/product_bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pytrends transformers sentencepiece scikit-learn beautifulsoup4 requests pandas matplotlib --quiet

In [ ]:
!pip install googlesearch-python pytrends transformers scikit-learn beautifulsoup4 requests --quiet

import time
import re
import math
import requests
import numpy as np
import pandas as pd
from pytrends.request import TrendReq
from transformers import pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LinearRegression
from bs4 import BeautifulSoup
from datetime import datetime
from googlesearch import search

# Initialize sentiment analyzer (Hugging Face)
sentiment_analyzer = pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english")

# -------------------------
# Utilities: web helpers
# -------------------------
HEADERS = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0 Safari/537.36"}

def safe_get(url, timeout=8):
    try:
        r = requests.get(url, headers=HEADERS, timeout=timeout)
        if r.status_code == 200:
            return r.text
        return ""
    except Exception:
        return ""

def extract_text_blocks(html, min_len=40, max_len=600):
    soup = BeautifulSoup(html, "html.parser")
    for s in soup(["script", "style", "noscript", "header", "footer", "svg", "form", "button"]):
        s.extract()
    text = soup.get_text(separator="\n")
    parts = [p.strip() for p in re.split(r'\n{2,}', text) if min_len <= len(p.strip()) <= max_len]
    return parts

# -------------------------
# Auto-collect reviews from top search results
# -------------------------
PREFERRED_DOMAINS = ["amazon.", "flipkart.", "walmart.", "reddit.", "trustpilot.", "bestbuy.", "consumerreports.", "thewirecutter.", "cnet.", "gsmarena.", "techradar.", "pcmag.", "producthunt.", "play.google", "apps.apple"]

def auto_collect_reviews(product_name, max_pages=8, per_page_limit=30):
    queries = [
        f"{product_name} reviews",
        f"{product_name} review",
        f"{product_name} complaints",
        f"buy {product_name}",
        f"{product_name} customer reviews",
    ]
    seen_urls = set()
    collected = []
    likely_product_found = False

    for q in queries:
        try:
            for url in search(q, num=max_pages, stop=max_pages, pause=1.0):
                if url in seen_urls:
                    continue
                seen_urls.add(url)
                lower = url.lower()
                if any(dom in lower for dom in ["amazon.", "flipkart.", "walmart.", "bestbuy.", "producthunt.", "play.google", "apps.apple"]):
                    likely_product_found = True
                html = safe_get(url)
                if not html:
                    continue
                blocks = extract_text_blocks(html)
                scored = []
                for b in blocks:
                    score = 0
                    lower_b = b.lower()
                    if any(w in lower_b for w in ["review", "rating", "stars", "customer", "customers", "complaint", "issue", "problem", "love", "hate"]):
                        score += 2
                    if len(b.split()) > 8:
                        score += 1
                    scored.append((score, b))
                scored.sort(key=lambda x: x[0], reverse=True)
                selected = [b for s,b in scored if s>0][:per_page_limit]
                if not selected:
                    selected = blocks[:min(len(blocks), per_page_limit)]
                for blk in selected:
                    if blk and blk not in collected:
                        collected.append(blk)
                time.sleep(0.8)
        except Exception:
            continue

    return collected[:400], likely_product_found

# -------------------------
# Google Trends helpers
# -------------------------
def get_trend_data(keyword, timeframe='today 12-m', geo=''):
    try:
        pytrends = TrendReq(hl='en-US', tz=360)
        pytrends.build_payload([keyword], timeframe=timeframe, geo=geo)
        data = pytrends.interest_over_time()
        if data.empty:
            return None
        data = data.reset_index()
        if 'isPartial' in data.columns:
            data = data.drop(columns=['isPartial'])
        return data
    except Exception:
        return None

def compute_trend_slope(df):
    if df is None or df.shape[0] < 3:
        return 0.0
    x = np.arange(len(df)).reshape(-1,1)
    y = df.iloc[:,1].values.reshape(-1,1)
    lr = LinearRegression().fit(x, y)
    slope = float(lr.coef_[0][0])
    return slope

def get_top_regions(keyword, timeframe='today 12-m'):
    try:
        pytrends = TrendReq(hl='en-US', tz=360)
        pytrends.build_payload([keyword], timeframe=timeframe)
        df_regions = pytrends.interest_by_region(resolution='COUNTRY', inc_low_vol=True, inc_geo_code=False)
        if df_regions is None or df_regions.empty:
            return []
        df_regions = df_regions.sort_values(by=keyword, ascending=False)
        top = df_regions[df_regions[keyword] > 0].head(10)
        return list(top.index[:5])
    except Exception:
        return []

# -------------------------
# Review analysis
# -------------------------
def analyze_reviews(review_texts, top_k_keywords=8):
    if not review_texts:
        return {"count": 0, "avg_sentiment_score": 0.0, "pos_pct": 0.0, "neg_pct": 0.0, "top_negative_keywords": []}
    sentiments = []
    for txt in review_texts:
        try:
            out = sentiment_analyzer(txt[:512])
            label = out[0]['label']
            score = out[0]['score']
            signed = score if label == "POSITIVE" else -score
            sentiments.append(signed)
        except Exception:
            sentiments.append(0.0)
    avg_score = float(np.mean(sentiments))
    pos_pct = float(sum(1 for s in sentiments if s > 0)/len(sentiments))*100
    neg_pct = float(sum(1 for s in sentiments if s < 0)/len(sentiments))*100

    neg_texts = [t for s,t in zip(sentiments, review_texts) if s < 0]
    top_neg_keywords = []
    if neg_texts:
        try:
            vect = TfidfVectorizer(stop_words='english', ngram_range=(1,2), max_features=2000)
            X = vect.fit_transform(neg_texts)
            sums = np.asarray(X.sum(axis=0)).ravel()
            terms = np.array(vect.get_feature_names_out())
            top_idx = sums.argsort()[::-1][:top_k_keywords]
            top_neg_keywords = list(terms[top_idx])
        except Exception:
            top_neg_keywords = []
    return {"count": len(review_texts), "avg_sentiment_score": avg_score, "pos_pct": pos_pct, "neg_pct": neg_pct, "top_negative_keywords": top_neg_keywords}

# -------------------------
# Success scoring & reasoning
# -------------------------
def compute_success_score(trend_slope, avg_sentiment, review_count, price_competitiveness_score):
    slope_score = (1 / (1 + math.exp(-10*(trend_slope/5)))) if trend_slope != 0 else 0.5
    sentiment_score = (avg_sentiment + 1) / 2
    review_score = min(review_count / 100.0, 1.0)
    price_score = price_competitiveness_score
    score = 0.4*slope_score + 0.35*sentiment_score + 0.15*review_score + 0.1*price_score
    components = {"slope_score": slope_score, "sentiment_score": sentiment_score, "review_score": review_score, "price_score": price_score}
    return score, components

def generate_failure_reason_and_mods(refined_input, analysis, components):
    reasons = []
    suggestions = []
    if analysis['count'] == 0:
        reasons.append("Insufficient real-user feedback or reviews.")
        suggestions.append("Collect early user trials, pilot test, gather feedback.")
    if analysis['neg_pct'] > 40:
        reasons.append("High negative sentiment.")
        suggestions.append("Address top complaints: " + ", ".join(analysis['top_negative_keywords'][:5]))
    if components['slope_score'] < 0.4:
        reasons.append("Low trend interest.")
        suggestions.append("Refine positioning or target trending adjacent categories.")
    if components['price_score'] < 0.4:
        reasons.append("Price competitiveness is low.")
        suggestions.append("Consider value-based pricing or cheaper variants.")
    if components['review_score'] < 0.2:
        reasons.append("Few reviews; adoption risk.")
        suggestions.append("Run early campaigns to gather reviews.")
    return reasons, suggestions

def generate_success_strategy(product_name, product_desc, top_regions, trend_slope, sentiment_analysis):
    slope = trend_slope
    if slope > 0.6:
        timeline = "1-4 weeks (rapid seeding)"
    elif slope > 0.15:
        timeline = "2-8 weeks (targeted campaigns)"
    else:
        timeline = "1-3 months (build awareness)"
    channels = ["Social media","Influencer partnerships"] if sentiment_analysis['pos_pct']>60 else ["Content marketing","Niche forums","Paid search"]
    regions = top_regions[:3] if top_regions else ["US","India","UK"]
    mods = []
    if sentiment_analysis['neg_pct']>30:
        mods.append("Fix top negative areas: " + ", ".join(sentiment_analysis['top_negative_keywords'][:5]))
    strategy = {
        "timeline": timeline,
        "channels": channels,
        "regions": regions,
        "key_recommendations": [
            "Create 5 short-form videos per top region",
            "Run small A/B tests for creative",
            "Collect early feedback and iterate on top pain points"
        ] + mods
    }
    return strategy

# -------------------------
# AUTO-MODE main function
# -------------------------
def run_product_assessor_auto():
    print("=== TrendScope AI — Auto Product Success Predictor (Auto Review Mode) ===")
    while True:
        product_name = input("\nProduct name (or 'quit' to exit): ").strip()
        if not product_name:
            print("Please enter a product name.")
            continue
        if product_name.lower() in ['quit','exit','q']:
            print("Exiting.")
            break

        # -------------------------
        # Auto-collect reviews (check idea availability first)
        # -------------------------
        print("\n🔎 Checking if similar product already exists online...")
        _, product_found = auto_collect_reviews(product_name, max_pages=4, per_page_limit=5)
        if product_found:
            print("⚠️ Idea already taken / similar product exists! Please try a different product idea.")
            continue  # go back to product name prompt
        else:
          print("🆕 This is a unique product idea!\n")
        # proceed if idea not found
        product_desc = input("Short product description / uses (1-2 sentences):\n> ").strip()
        target_users = input("Target users (comma-separated, optional):\n> ").strip()
        price_input = input("Is the product priced competitively? (0..1) [Enter if unknown]:\n> ").strip()
        try:
            price_score = float(price_input) if price_input else 0.6
            price_score = max(0.0, min(1.0, price_score))
        except:
            price_score = 0.6

        print("\n🔎 Collecting detailed reviews from web...")
        reviews, _ = auto_collect_reviews(product_name, max_pages=8)
        print(f"Collected {len(reviews)} review-like text blocks.")

        # Trend data
        print("\nFetching Google Trends data...")
        trend_df = get_trend_data(product_name)
        slope = compute_trend_slope(trend_df) if trend_df is not None else 0.0
        top_regions = get_top_regions(product_name)
        print(f"Trend slope: {slope:.4f}, top regions: {top_regions[:5]}")

        # Reviews analysis
        print("\nAnalyzing reviews and sentiment...")
        review_analysis = analyze_reviews(reviews)
        print("Avg sentiment:", review_analysis['avg_sentiment_score'], "Positive%:", review_analysis['pos_pct'], "Negative%:", review_analysis['neg_pct'])

        # Compute success score
        score, comps = compute_success_score(slope, review_analysis['avg_sentiment_score'], review_analysis['count'], price_score)
        print("\n=== Prediction Summary ===")
        print(f"Composite success score: {score*100:.1f}%")

        threshold = 0.5
        if score < threshold:
            print("\nPrediction: Likely unsuccessful.")
            reasons, suggestions = generate_failure_reason_and_mods({"name": product_name, "desc": product_desc, "users": target_users}, review_analysis, comps)
            print("\nReasons:")
            for r in reasons: print("-", r)
            print("\nSuggestions:")
            for s in suggestions: print("-", s)
        else:
            print("\nPrediction: Likely to succeed (with proper strategy).")
            strategy = generate_success_strategy(product_name, product_desc, top_regions, slope, review_analysis)
            print("\nRecommended launch timeline:", strategy['timeline'])
            print("Top regions:", ", ".join(strategy['regions']))
            print("Best channels:")
            for ch in strategy['channels']: print("-", ch)
            print("Key action items:")
            for k in strategy['key_recommendations']: print("-", k)
            if review_analysis['top_negative_keywords']:
                print("\nSuggested product modifications (based on negative feedback):")
                for kw in review_analysis['top_negative_keywords'][:8]:
                    print("-", kw)
            print("\nProduct description:")
            print(product_desc)
        again = input("\nEvaluate another product? (yes/no): ").strip().lower()
        if not again.startswith('y'):
            print("Done. Exiting.")
            break

# Run the auto-mode assessor
run_product_assessor_auto()

Device set to use cuda:0


=== TrendScope AI — Auto Product Success Predictor (Auto Review Mode) ===

Product name (or 'quit' to exit): kitchen robot

🔎 Checking if similar product already exists online...
🆕 This is a unique product idea!

Short product description / uses (1-2 sentences):
> the kitchen robot is a robot which first observes the work in the kitchen i,e food recipies, utensils cleaning and also kitchen platform,stove cleaning and also cleans the dirt on the kitchen walls
Target users (comma-separated, optional):
> house maids, house wives
Is the product priced competitively? (0..1) [Enter if unknown]:
> 

🔎 Collecting detailed reviews from web...
Collected 0 review-like text blocks.

Fetching Google Trends data...


/usr/local/lib/python3.12/dist-packages/pytrends/request.py:260: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.fillna(False)


Trend slope: 0.9339, top regions: ['Guam', 'Haiti', 'Slovenia', 'Czechia', 'Slovakia']

Analyzing reviews and sentiment...
Avg sentiment: 0.0 Positive%: 0.0 Negative%: 0.0

=== Prediction Summary ===
Composite success score: 58.1%

Prediction: Likely to succeed (with proper strategy).

Recommended launch timeline: 1-4 weeks (rapid seeding)
Top regions: Guam, Haiti, Slovenia
Best channels:
- Content marketing
- Niche forums
- Paid search
Key action items:
- Create 5 short-form videos per top region
- Run small A/B tests for creative
- Collect early feedback and iterate on top pain points

Product description:
the kitchen robot is a robot which first observes the work in the kitchen i,e food recipies, utensils cleaning and also kitchen platform,stove cleaning and also cleans the dirt on the kitchen walls

Evaluate another product? (yes/no): no
Done. Exiting.
